In [2]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn
import dataframe_image as dfi
import math

# define report path
Path("report").mkdir(parents=True, exist_ok=True)

SETUP

In [3]:

resolution_list = ['1D','1H']

# report_heading = "Utilities 1Y"  # change for the report heading
report_heading = "COINS_RESUME"   # change for the report heading

pos_value = 7500 # x2 for pair buy & sell
mar_req = 0.3 # margin requirement
cash_req = mar_req * pos_value * 2 # required cash
avg_slip = 0.001 # per order per position value
avg_commission = 0.007 # per contract

avg_fin_cost_dic = {'1D':(0.07/365),'1H':(0.07/365/7)} # per borrowed cash per day&hour

In [4]:
max_items = 24
pairs_dic_filtered = {}

with open('pairs_final.pkl', 'rb') as f:
    pairs_dic = pickle.load(f)

i = 0
for key,item in pairs_dic.items():
    if i<max_items:
        i = i+1
        pairs_dic_filtered[key] = item
        
pairs_dic = pairs_dic_filtered
pairs_dic

{('NEARUSDT', 'TRXUSDT'): 0.03,
 ('NEARUSDT', 'XRPUSDT'): 0.18,
 ('LINKUSDT', 'XRPUSDT'): 0.06,
 ('ADAUSDT', 'NEOUSDT'): 27.67,
 ('ADAUSDT', 'TRXUSDT'): 0.18,
 ('DOTUSDT', 'TRXUSDT'): 0.01,
 ('SOLUSDT', 'TRXUSDT'): 0.0,
 ('BTCUSDT', 'TRXUSDT'): 0.0,
 ('NEOUSDT', 'TRXUSDT'): 0.01,
 ('LINKUSDT', 'TRXUSDT'): 0.01,
 ('DOGEUSDT', 'TRXUSDT'): 0.79,
 ('DOTUSDT', 'NEARUSDT'): 0.35,
 ('ETHUSDT', 'TRXUSDT'): 0.0,
 ('NEOUSDT', 'SOLUSDT'): 2.07}

In [5]:
max_drop_items = 10
drop_dic_filtered = {}

with open('pairs_dropped.pkl', 'rb') as f:
    drop_dic = pickle.load(f)  
drop_dic

i = 0
for key,item in drop_dic.items():
    if i<max_drop_items:
        i = i+1
        drop_dic_filtered[key] = item
        
drop_dic = drop_dic_filtered
drop_dic

{('SOLUSDT', 'XRPUSDT'): 0.02}

DEFINE STRATEGIES

In [6]:
# buy and sell at upper and lower bollinger band, exit at sma
def exit_sma(data, lower_bb, upper_bb, sma_20, _):
    buy_price = []
    sell_price = []
    bb_signal = []
    position = []
    signal = 0
      
    position = [0] * len(data)
      
    for i in range(len(data)):
        # define band crossings
        if data[i - 1] > lower_bb[i - 1] and data[i] < lower_bb[i]:
            if position[i-1] == 0:
                buy_price.append(data[i])
                sell_price.append(np.nan)
                signal = 1
                bb_signal.append(signal)
                position[i] = 1
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]
        elif data[i - 1] < upper_bb[i - 1] and data[i] > upper_bb[i]:
            if position[i-1] == 0:
                buy_price.append(np.nan)
                sell_price.append(data[i])
                signal = -1
                bb_signal.append(signal)
                position[i] = -1
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]

        # define sma crossings
        elif data[i - 1] > sma_20[i - 1] and data[i] < sma_20[i]:
            if position[i-1] != 0:
                buy_price.append(data[i])
                sell_price.append(np.nan)
                signal = 1
                bb_signal.append(signal)
                position[i] = 0
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]
        elif data[i - 1] < sma_20[i - 1] and data[i] > sma_20[i]:
            if position[i-1] != 0:
                buy_price.append(np.nan)
                sell_price.append(data[i])
                signal = -1
                bb_signal.append(signal)
                position[i] = 0
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]

        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            bb_signal.append(0)
            position[i] = position[i-1]

    return buy_price, sell_price, bb_signal, position


# buy and sell at upper and lower bollinger band, keep position, no exit
def no_exit(data, lower_bb, upper_bb, *_):

    buy_price = []
    sell_price = []
    bb_signal = []
    position = []
    signal = 0
           
    position = [0] * len(data)
    
    for i in range(len(data)):
        if data[i - 1] > lower_bb[i - 1] and data[i] < lower_bb[i]:
            if signal != 1:
                buy_price.append(data[i])
                sell_price.append(np.nan)
                signal = 1
                bb_signal.append(signal)
                position[i] = 1
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]
        elif data[i - 1] < upper_bb[i - 1] and data[i] > upper_bb[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(data[i])
                signal = -1
                bb_signal.append(signal)
                position[i] = -1
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            bb_signal.append(0)
            position[i] = position[i-1]

    return buy_price, sell_price, bb_signal, position


# buy and sell at upper and lower bollinger band, exit at sma, no exit if moving towards n days-sma
def skp_sma(data, lower_bb, upper_bb, sma_20, sma_nd):
    buy_price = []
    sell_price = []
    bb_signal = []
    position = []
    signal = 0
      
    position = [0] * len(data)
      
    for i in range(len(data)):
        # define band crossings
        if data[i - 1] > lower_bb[i - 1] and data[i] < lower_bb[i]:
            if position[i-1] != 1:
                buy_price.append(data[i])
                sell_price.append(np.nan)
                signal = 1
                bb_signal.append(signal)
                position[i] = 1
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]
        elif data[i - 1] < upper_bb[i - 1] and data[i] > upper_bb[i]:
            if position[i-1] != -1:
                buy_price.append(np.nan)
                sell_price.append(data[i])
                signal = -1
                bb_signal.append(signal)
                position[i] = -1
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]

        # define sma crossings
        elif data[i - 1] > sma_20[i - 1] and data[i] < sma_20[i]:
            if data[i] < sma_nd[i]: # check 20d-sma
                if position[i-1] == -1:
                    buy_price.append(data[i])
                    sell_price.append(np.nan)
                    signal = 1
                    bb_signal.append(signal)
                    position[i] = 0
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    bb_signal.append(0)
                    position[i] = position[i-1]
                    
            else: # moving towards 20d-sma               
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]
                
                
        elif data[i - 1] < sma_20[i - 1] and data[i] > sma_20[i]:
            if data[i] > sma_nd[i]: # check 20d-sma
                if position[i-1] == 1:
                    buy_price.append(np.nan)
                    sell_price.append(data[i])
                    signal = -1
                    bb_signal.append(signal)
                    position[i] = 0
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    bb_signal.append(0)
                    position[i] = position[i-1]
                    
            else: # moving towards 20d-sma               
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
                position[i] = position[i-1]

        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            bb_signal.append(0)
            position[i] = position[i-1]

    return buy_price, sell_price, bb_signal, position

DEFINE SMA & BOLLINGER FUNCTIONS

In [7]:
def sma(values, n):
    return pd.Series(values).rolling(n).mean()

def bb(data, sma, sd=2.0, n=20):
    std = data.rolling(n).std(ddof=1) # default ddof=1, sample standard deviation, divide by (n-1)
    bb_up = sma + std * sd
    bb_low = sma - std * sd
    return bb_up, bb_low

CALCULATE STRATEGY RETURNS

In [8]:
def calculate_returns(data_dic, str_list, b=2):  # b is 1.0, 1.5, 2.0, 2.5 or 3 std

    # list of strategies to apply
    strategy_list = str_list

    # create a dataframe to fill with strategy return values
    df = pd.DataFrame.from_dict(data_dic, orient="index")
    returns_df = df.iloc[:, 1:]  # drop the first column
    returns_with_costs_df = df.iloc[:, 1:]
    # create a dataframe to fill order details
    orders_df = df.iloc[:, 1:]

    for res in resolution_list:

        for st in range(len(strategy_list)):
            apply_strategy = globals()[strategy_list[st]]

            returns_list = []
            costs_list = []
            orders_list = []

            for key in data_dic:
                ticker1 = key[0]
                ticker2 = key[1]
                hedge = data_dic[key]
                # print(f"EQUATION: {ticker2}={hedge}*{ticker1}")

                # read data from csv
                alltickersfile = "data/tickers_" + res + ".csv"
                df = pd.read_csv(alltickersfile)
                df_sorted = df.set_index(["ticker", "time"]).sort_index()  # set indexes
                df1 = df_sorted.xs(ticker1)  # the first ticker
                df2 = df_sorted.xs(ticker2)  # the second ticker

                # calculate spread
                df1_1 = hedge * df1
                df_spread = df2.subtract(df1_1).round(5)

                # add sma & bollinger bands
                df_spread["sma_20"] = sma(df_spread.Close, 20)
                
                if res == '1D':                 
                    df_spread['sma_50d']=sma(df_spread.Close, 50) # add 50d sma for 1D resolution
                
                else:
                    df_spread['sma_20d']=sma(df_spread.Close, 20*7) # add 20d sma for 1H resolution
                
                for i in range(2, 7):
                    step = 0.5
                    b_band = round(step*i,1)
                    (
                        df_spread["bb_up_" + str(b_band)],
                        df_spread["bb_low_" + str(b_band)],
                    ) = bb(df_spread["Close"], df_spread["sma_20"], b_band, 20)

                df_spread_bb = df_spread.copy()
                df_spread_bb = df_spread_bb.dropna()

                # apply strategy & positions
                
                if res == '1D':               
                    buy_price, sell_price, signal, position = apply_strategy(
                        df_spread_bb["Close"],
                        df_spread_bb["bb_low_" + str(round(b,1))],
                        df_spread_bb["bb_up_" + str(round(b,1))],
                        df_spread_bb["sma_20"],
                        df_spread_bb["sma_50d"]
                    )
                else:                       
                    buy_price, sell_price, signal, position = apply_strategy(
                        df_spread_bb["Close"],
                        df_spread_bb["bb_low_" + str(round(b,1))],
                        df_spread_bb["bb_up_" + str(round(b,1))],
                        df_spread_bb["sma_20"],
                        df_spread_bb["sma_20d"]
                    )
                        
                signal_df = (
                    pd.DataFrame(signal)
                    .rename(columns={0: "signal"})
                    .set_index(df_spread_bb.index)
                )
                position_df = (
                    pd.DataFrame(position)
                    .rename(columns={0: "bb_position"})
                    .set_index(df_spread_bb.index)
                )

                # calculate price difference per bar
                df_spread_bb_ret = pd.DataFrame(np.diff(df_spread_bb["Close"])).rename(
                    columns={0: "returns"}
                )

                # calculate return per bar
                bb_strategy_ret = []
                for i in range(len(df_spread_bb_ret)):
                    try:
                        returns = (
                            df_spread_bb_ret["returns"][i]
                            * position_df["bb_position"][i]
                        )
                        bb_strategy_ret.append(returns)
                    except:
                        pass

                bb_strategy_ret_df = pd.DataFrame(bb_strategy_ret).rename(
                    columns={0: "bb_returns"}
                )

                # calculate investment return
                contract_price = round(df2.iloc[-1,df2.columns.get_loc('Close')],0) # get the latest price
                number_of_stocks = math.floor(pos_value / contract_price)
                bb_investment_ret = []

                for i in range(len(bb_strategy_ret_df["bb_returns"])):
                    returns = number_of_stocks * bb_strategy_ret_df["bb_returns"][i]
                    bb_investment_ret.append(returns)

                bb_investment_ret_df = pd.DataFrame(bb_investment_ret).rename(
                    columns={0: "investment_returns"}
                )
                total_investment_ret = round(
                    sum(bb_investment_ret_df["investment_returns"]), 2
                )
                profit_percentage = math.floor((total_investment_ret / cash_req) * 100)

                returns_list.append(profit_percentage)
                                               
                # get orders & order returns
                cum_inv_ret = bb_investment_ret_df.cumsum()
                o_df = signal_df[['signal']].reset_index()
                o_df['cum_return'] = cum_inv_ret
                mask = o_df.signal!=0
                o_df = o_df[mask][['signal','cum_return']]
                o_df = o_df.reset_index().dropna()
                o_df['prof_loss'] = o_df.cum_return.diff()

                if not o_df.empty:
                    o_df.at[0, 'prof_loss'] = o_df['cum_return'][0]
                    o_df.at[len(o_df), 'prof_loss'] = total_investment_ret - o_df.loc[o_df.index[-1],'cum_return']
                    o_df.at[len(o_df)-1, 'cum_return'] = total_investment_ret
                    orders_list.append(o_df.prof_loss)
                else:
                    orders_list.append([])
                    
                # get costs
                slip = signal_df.signal.abs() * avg_slip * pos_value
                commission = signal_df.signal.abs() * avg_commission * number_of_stocks * 2
                fincost = (
                    position_df.bb_position.abs()
                    * avg_fin_cost_dic[res]
                    * (2 * pos_value - cash_req)
                )

                total_cost = (
                    slip.sum().round(1)
                    + commission.sum().round(1)
                    + fincost.sum().round(1)
                )

                total_investment_ret_cost = (
                    round(sum(bb_investment_ret_df["investment_returns"]), 2)
                    - total_cost
                )
                profit_percentage_cost = math.floor(
                    (total_investment_ret_cost / cash_req) * 100
                )

                costs_list.append(profit_percentage_cost)

            returns_df[res + "_" + strategy_list[st]] = returns_list
            returns_with_costs_df[res + "_" + strategy_list[st]] = costs_list
            orders_df[res + "_" + strategy_list[st]] = orders_list

    return returns_df, returns_with_costs_df, orders_df

In [11]:
from IPython.display import display_html 

strategy_list = ["exit_sma", "no_exit", "skp_sma"]
str_list = ["exit_sma", "no_exit"]

df1_2_0std, df2_2_0std, orders_2_0std = calculate_returns(pairs_dic, str_list, b=2.0) # b is 1.0, 1.5, 2.0, 2.5 or 3 std

for col in df1_2_0std.columns:
    print(f"average return - {col} : {round(df1_2_0std[col].mean(),2)}%")

for col in df2_2_0std.columns:
    print(f"average return (costs included) - {col} : {round(df2_2_0std[col].mean(),2)}%")
    
df1_sort = df1_2_0std.copy()
df2_sort = df2_2_0std.copy()
df1_sort['mean'] = df1_2_0std.mean(axis=1)
df1_sort = df1_sort.sort_values('mean', ascending=False)
df2_sort['mean'] = df2_2_0std.mean(axis=1)
df2_sort = df2_sort.sort_values('mean', ascending=False)
df1_sort = df1_sort.drop(columns=['mean'])
df2_sort = df2_sort.drop(columns=['mean'])

df1_style = df1_sort.style.set_table_attributes("style='display:inline; margin-right:20px;'").set_caption("2DEV Return%")
df2_style = df2_sort.style.set_table_attributes("style='display:inline'").set_caption(" 2DEV Return% (costs inc.)")

display_html(df1_style._repr_html_() + df2_style._repr_html_(), raw=True)


fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14, 7))
fig.tight_layout()
seaborn.heatmap(
    df1_sort,
    xticklabels=df1_sort.columns,
    yticklabels=df1_sort.index,
    cmap="Greens",
    square=True,
    vmin=0,
    ax=ax1
)

seaborn.heatmap(
    df2_sort,
    xticklabels=df2_sort.columns,
    yticklabels=df2_sort.index,
    cmap="Greens",
    square=True,
    vmin=0,
    ax=ax2
)

dpi_set=100
dfi.export(df1_style, 'summary1.png', dpi=dpi_set)
dfi.export(df2_style, 'summary2.png', dpi=dpi_set)
plt.savefig('summary_plot1', dpi=dpi_set, bbox_inches='tight', pad_inches=0)

KeyError: 'NEARUSDT'

In [12]:
max_pairs = 5

col_len = len(orders_2_0std.columns)

fig, axs = plt.subplots(1,col_len, figsize=(4*col_len, 20))
# _ = plt.figure(figsize=(5,25))

for x in range(col_len):
    vip =seaborn.violinplot(data=orders_2_0std[orders_2_0std.columns[x]][:max_pairs], inner=None, orient='h', ax=axs[x]);
    swp =seaborn.swarmplot(data=orders_2_0std[orders_2_0std.columns[x]][:max_pairs], size=7, color="k", alpha=0.5, orient='h', ax=axs[x]);
    vip.set(yticklabels=[])
    swp.set(yticklabels=[])
    
    if x ==0:
        vip.set_yticklabels(list(orders_2_0std.index.values)[:max_pairs])
    
for ax in range(len(axs)):
    axs[ax].set_title(orders_2_0std.columns[ax], fontstyle='italic')

fig.text(0.5, 0.10, 'PROFIT/LOSS ($ per trade ●)', ha='center', va='center')

dpi_set=100
fig.savefig('summary_plot2', dpi=dpi_set, bbox_inches='tight', pad_inches=0)

NameError: name 'orders_2_0std' is not defined